In [1]:
from collections import defaultdict as dd
import os
import json
import sqlite3
import ast
import re
from sqlite3 import Error
from collections import Counter

import sys
sys.path.append("rat-sql")

In [2]:
def read_json(file):
    with open(file) as json_file:
        json_data = json.load(json_file)
    
    return json_data

def read_jsonl(file):
    with open(file) as json_file:
        json_data = [json.loads(line) for line in json_file]
    
    return json_data

In [3]:
exp = 'bs=12,lr=7.4e-04,bert_lr=3.0e-06,end_lr=0e0,att=1'
ratsql_infereval_loc = f'rat-sql/logdir/bert_run/{exp}/ie_dirs/'
ratsql_train = 'rat-sql/data/spider/train_spider.json'
ratsql_train_others = 'rat-sql/data/spider/train_others.json'
# step = [1] + [(5000 * x) + 100 for x in range(1, 17)] + [81000]
step = 5100
ratsql_infer = os.path.join(ratsql_infereval_loc, f'bert_run_true_1-step{step}.infer')
ratsql_eval  = os.path.join(ratsql_infereval_loc, f'bert_run_true_1-step{step}.eval')

In [4]:
ratsql_infer

'rat-sql/logdir/bert_run/bs=12,lr=7.4e-04,bert_lr=3.0e-06,end_lr=0e0,att=1/ie_dirs/bert_run_true_1-step5100.infer'

In [8]:
ratsql_eval  = os.path.join(ratsql_infereval_loc, f'bert_run_true_1-step{step}.eval')

evaluation = read_jsonl(ratsql_eval)

In [22]:
exact_match = {'easy': 0, 'medium': 0 , 'hard': 0, 'extra': 0}

for item in evaluation[0]['per_item']:
    # print(f"Hardness: {item['hardness']}")
    # print(f"Exact: {item['exact']}")

    if item['exact']:
        exact_match[item['hardness']] += 1

print(exact_match)

{'easy': 172, 'medium': 238, 'hard': 76, 'extra': 63}


In [23]:
for item in evaluation[0]['per_item']:
    hardness_cnt[item['hardness']] += 1

hardness_cnt

{'easy': 248, 'medium': 446, 'hard': 174, 'extra': 166}

In [35]:
exp = 'bs=12,lr=7.4e-04,bert_lr=3.0e-06,end_lr=0e0,att=1'

steps = [1] + [(5000 * x) + 100 for x in range(1, 17)] + [81000]
step_result = {}
for step in steps:
    ratsql_eval  = os.path.join(ratsql_infereval_loc, f'bert_run_true_1-step{step}.eval')

    evaluation = read_jsonl(ratsql_eval)

    hardness_cnt = {'easy': 248, 'medium': 446, 'hard': 174, 'extra': 166}

    exact_match = {'easy': 0, 'medium': 0 , 'hard': 0, 'extra': 0}
    overall = 0
    for item in evaluation[0]['per_item']:
        if item['exact']:
            exact_match[item['hardness']] += 1
            overall += 1

    print(f"Step: {step} Result")

    step_result[step] = {}
    for hardness in hardness_cnt.keys():
        step_result[step][hardness] = round(exact_match[hardness] / hardness_cnt[hardness], 3) * 100
        print(f"{hardness}: {exact_match[hardness] / hardness_cnt[hardness]}")
    step_result[step]['overall'] = round(overall / 1034, 3) * 100

    print(f"Overall: {overall / 1034}")

    

    print('='*30)

fileName = f'{exp}_result.json'

with open(fileName, 'w') as fp:
    json.dump(step_result, fp, indent=4)

Step: 1 Result
easy: 0.0
medium: 0.0
hard: 0.0
extra: 0.0
Overall: 0.0
Step: 5100 Result
easy: 0.6935483870967742
medium: 0.5336322869955157
hard: 0.4367816091954023
extra: 0.3795180722891566
Overall: 0.5309477756286267
Step: 10100 Result
easy: 0.7782258064516129
medium: 0.5739910313901345
hard: 0.4885057471264368
extra: 0.37349397590361444
Overall: 0.5764023210831721
Step: 15100 Result
easy: 0.8225806451612904
medium: 0.6188340807174888
hard: 0.5459770114942529
extra: 0.42168674698795183
Overall: 0.6237911025145068
Step: 20100 Result
easy: 0.06854838709677419
medium: 0.03811659192825112
hard: 0.005747126436781609
extra: 0.012048192771084338
Overall: 0.035783365570599614
Step: 25100 Result
easy: 0.7338709677419355
medium: 0.4798206278026906
hard: 0.3045977011494253
extra: 0.16265060240963855
Overall: 0.46034816247582205
Step: 30100 Result
easy: 0.7741935483870968
medium: 0.547085201793722
hard: 0.5114942528735632
extra: 0.3433734939759036
Overall: 0.562862669245648
Step: 35100 Result
e